## Dependencies

---

## Pre-loading

---

In [2]:
import pandas as pd

### Functions:

In [3]:
def create_dim(column:str, dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the feature, parse it to a dataframe and add a new column with an unique ID.
    :param column: feature to extract.
    :param dataframe: object where feature will be extracted.
    :return: dataframe with the feature and its new column.
    """
    dim = dataframe[f"{column}"].to_frame()
    dim[f"{column}_id"] = dim.index + 1
    return dim

### Dimensions & facts: